<a href="https://colab.research.google.com/github/abhi082001/abhi082001/blob/main/Main_implementation_RERC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Train Example

In [ ]:
!pip install jsonlines

In [ ]:
import json

with open('test.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)

article = train_data[0]
article

{'_id': '8a65901a0bdb11eba7f7acde48001122',
 'type': 'compositional',
 'question': 'Which country the director of film One Law For The Woman is from?',
 'context': [['Michael Govan',
   ['Michael Govan( born 1963) is the director of the Los Angeles County Museum of Art since 2006.',
    'Prior to this, Govan worked as the director of the Dia Art Foundation in New York City.']],
  ['John Donatich',
   ['John Donatich is the Director of Yale University Press.']],
  ['Peter Levin',
   ['Peter Levin is an American director of film, television and theatre.']],
  ['Ian Barry (director)',
   ['Ian Barry is an Australian director of film and TV.']],
  ['John Farrell (businessman)',
   ['John Farrell is the director of YouTube in Latin America.']],
  ['One Law for the Woman',
   ['One Law for the Woman is a 1924 American silent western film directed by Dell Henderson and starring Cullen Landis, Mildred Harris and Cecil Spooner.']],
  ['Dana Blankstein',
   ['Dana Blankstein- Cohen( born March 3

In [ ]:
with open(r'C:\Users\Abhishek\Desktop\TCS Work\Code_notebooks\datasets\data_ids_april7\test.json') as f:
    test_data = json.load(f)

test = test_data[0]
test

{'_id': '8a65901a0bdb11eba7f7acde48001122',
 'type': 'compositional',
 'question': 'Which country the director of film One Law For The Woman is from?',
 'context': [['Michael Govan',
   ['Michael Govan( born 1963) is the director of the Los Angeles County Museum of Art since 2006.',
    'Prior to this, Govan worked as the director of the Dia Art Foundation in New York City.']],
  ['John Donatich',
   ['John Donatich is the Director of Yale University Press.']],
  ['Peter Levin',
   ['Peter Levin is an American director of film, television and theatre.']],
  ['Ian Barry (director)',
   ['Ian Barry is an Australian director of film and TV.']],
  ['John Farrell (businessman)',
   ['John Farrell is the director of YouTube in Latin America.']],
  ['One Law for the Woman',
   ['One Law for the Woman is a 1924 American silent western film directed by Dell Henderson and starring Cullen Landis, Mildred Harris and Cecil Spooner.']],
  ['Dana Blankstein',
   ['Dana Blankstein- Cohen( born March 3

## Relation Extractor

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00


In [ ]:
import torch
import copy
from torch import nn
import numpy as np
from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel, BertLayer
from transformers import (BertConfig, BertTokenizer, BertForQuestionAnswering,
                          RobertaConfig, RobertaTokenizer, RobertaForQuestionAnswering,
                          AlbertConfig, AlbertTokenizer, AlbertForQuestionAnswering)

import os
import re
import logging
import argparse
from tqdm import tqdm, trange
from numpy.random import shuffle

import numpy as np
import random
import json

In [ ]:

MODEL_CLASSES = {
    'bert': (BertConfig, BertForQuestionAnswering, BertTokenizer),
    'roberta': (RobertaConfig, RobertaForQuestionAnswering, RobertaTokenizer),
    'albert': (AlbertConfig, AlbertForQuestionAnswering, AlbertTokenizer),
}

IGNORE_INDEX = -100
dropout = torch.nn.Dropout(0.1)

Q_type = {"compositional":0, "inference":1, "comparison":2, "bridge_comparison":3}
relations = ['date of birth', 'publisher', 'child', 'director', 'composer',
            'doctoral advisor', 'mother', 'place of death', 'award received',
            'sibling', 'father', 'occupation', 'educated at', 'country of citizenship',
            'spouse', 'creator', 'cause of death', 'country of origin', 'performer',
            'employer', 'place of birth', 'place of detention', 'student of', 'publication date',
            'place of burial', 'inception', 'has part', 'manufacturer', 'presenter', 'country',
            'founded by', 'editor', 'producer', 'date of death', 'Birth-Death']
r_dict = {'date of birth': 0, 'publisher': 1, 'child': 2, 'director': 3, 'composer': 4,
            'doctoral advisor': 5, 'mother': 6, 'place of death': 7, 'award received': 8,
            'sibling': 9, 'father': 10, 'occupation': 11, 'educated at': 12,
            'country of citizenship': 13, 'spouse': 14, 'creator': 15, 'cause of death': 16,
            'country of origin': 17, 'performer': 18, 'employer': 19, 'place of birth': 20,
            'place of detention': 21, 'student of': 22, 'publication date': 23,
            'place of burial': 24, 'inception': 25, 'has part': 26, 'manufacturer': 27,
            'presenter': 28, 'country': 29, 'founded by': 30, 'editor': 31, 'producer': 32,
            'date of death': 33, 'Birth-Death': 34}


class MeanMaxPooling(nn.Module):
    def __init__(self):
        super(MeanMaxPooling, self).__init__()

    def forward(self, doc_state, mask):
        """
        :param doc_state:  N x L x d
        :param entity_lens:  N x L
        :return: N x 2d
        """
        max_pooled = torch.max(doc_state, dim=1)[0]
        mean_pooled = torch.sum(doc_state, dim=1) / torch.sum(mask, dim=1).unsqueeze(1)
        output = torch.cat([max_pooled, mean_pooled], dim=1)  # N x 2d
        return output

class BilinearSeqAttn(nn.Module):
    """A bilinear attention layer over a sequence X w.r.t y:
    * o_i = x_i'Wy for x_i in X.
    """
    def __init__(self, x_size, y_size, identity=False):
        super(BilinearSeqAttn, self).__init__()
        if not identity:
            self.linear = nn.Linear(y_size, x_size)
        else:
            self.linear = None

    def forward(self, x, y, x_mask):
        """
        x = [batch, len, h1]
        y = [batch, h2]
        x_mask = [batch, len]
        """
        x = dropout(x)
        y = dropout(y)

        Wy = self.linear(y) if self.linear is not None else y
        #xWy = x.bmm(Wy.unsqueeze(2)).squeeze(2)
        xWy = torch.matmul(x, Wy.unsqueeze(2)).squeeze(2)
        xWy = xWy + (1.0-x_mask)*(-10000.0)
        return xWy # [batch,len]

class GetSpanStartEnd(nn.Module):
    # supports MLP attention and GRU for pointer network updating
    def __init__(self, x_size, h_size, do_indep_attn=True, attn_type="Bilinear", do_ptr_update=True):
        super(GetSpanStartEnd, self).__init__()

        self.attn  = BilinearSeqAttn(x_size, h_size)
        self.attn2 = BilinearSeqAttn(x_size, h_size) if do_indep_attn else None

        self.rnn = nn.GRUCell(x_size, h_size) if do_ptr_update else None

    # x -- doc_hiddens [10,384,250]
    # h0 -- question_avg_hidden [10,125]
    # x_mask [10,384]
    def forward(self, x, h0, x_mask):
        """
        x = [batch, len, x_hidden_size]
        h0 = [batch, h_size]
        x_mask = [batch, len]
        """
        start_scores = self.attn(x, h0, x_mask) # [10,384]
        # start_scores [batch, len]

        if self.rnn is not None:
            ptr_net_in = torch.bmm(torch.nn.Softmax(dim=1)(start_scores).unsqueeze(1), x).squeeze(1) # [10,250]
            ptr_net_in = dropout(ptr_net_in)
            h0 = dropout(h0)
            h1 = self.rnn(ptr_net_in, h0) # [10,125]
            # h1 same size as h0
        else:
            h1 = h0

        end_scores = self.attn(x, h1, x_mask) if self.attn2 is None else self.attn2(x, h1, x_mask)
        # end_scores = batch * len
        return start_scores, end_scores # [10,384]


class BertForTypeSpanComp(BertPreTrainedModel):

    def __init__(self, config):
        super(BertForTypeSpanComp, self).__init__(config)
        self.bert = BertModel(config)
        self.encoder = BertLayer(config)
        self.pooler = MeanMaxPooling()
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.TypeLinear = torch.nn.Linear(config.hidden_size*2, 4)
        ptrnet = GetSpanStartEnd(config.hidden_size,config.hidden_size*2)
        self.RelaPointer = nn.ModuleList([copy.deepcopy(ptrnet) for _ in range(8)])
        self.init_weights()

    def forward(self, batch, is_train = False):
        input_ids, attention_mask = batch['context_idxs'], batch['context_mask']
        bsz = input_ids.size(0)
        embed_output = self.bert(input_ids, None, attention_mask)
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        encoder_output = self.encoder(embed_output[0], extended_attention_mask)[0]
        pooled_output = self.dropout(self.pooler(encoder_output,attention_mask))
        t_logits = self.TypeLinear(pooled_output)
        R_logits = []
        for layer_module in self.RelaPointer:
            start,end = layer_module(encoder_output, pooled_output, attention_mask)
            R_logits += [start,end]
        R_logits = torch.cat(R_logits,-1)
        r_logits = torch.matmul(R_logits.view(bsz, -1, 4), t_logits.view(-1,4,1)).view(bsz, -1, 4)

        r1s,r1e,r2s,r2e = r_logits.split(1, dim=-1)
        r1s,r1e,r2s,r2e = r1s.squeeze(-1), r1e.squeeze(-1), r2s.squeeze(-1), r2e.squeeze(-1)

        if is_train:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss1 = loss_fct(r1s, batch['y1s']) + loss_fct(r1e, batch['y1e'])
            loss2 = loss_fct(r2s, batch['y2s']) + loss_fct(r2e, batch['y2e'])
            loss = loss_fct(t_logits, batch['y']) + loss1 + loss2
            return loss, loss1, loss2
        else:
            return t_logits, r1s,r1e,r2s,r2e

In [ ]:


def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x


def find_pos(context, ans):
    context = context.replace(ans,'{'+ans+'}').split()
    y1,y2 = 0,0
    for i in range(len(context)):
        if '{' in context[i]:
            y1 = i
        if '}' in context[i]:
            y2 = i
            break
    return y1,y2


def F1(a,b,c,d):
    correct = max([0,min(b,d)-max(a,c)])
    if correct == 0:
        return 0.0,0.0,0.0
    pred, recall = correct/(b-a), correct/(d-c)
    return 2*pred*recall/(pred+recall), pred, recall




logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


class DataIterator(object):
    def __init__(self, example_file, batch_size, max_len, sequential=False, subset=False):
        with open(example_file, 'r', encoding='utf-8') as fh:
            self.examples = json.load(fh)
        self.bsz = batch_size
        self.sequential = sequential
        self.max_len = max_len
        self.example_ptr = 0
        if subset:
            self.examples = random.sample(self.examples, 0.2*len(self.examples))
        elif not sequential:
            shuffle(self.examples)

    def refresh(self):
        self.example_ptr = 0
        if not self.sequential:
            shuffle(self.examples)

    def empty(self):
        return self.example_ptr >= len(self.examples)

    def __len__(self):
        return int(np.ceil(len(self.examples)/self.bsz))

    def __iter__(self):
        # BERT input
        context_idxs = torch.LongTensor(self.bsz, self.max_len)
        context_mask = torch.LongTensor(self.bsz, self.max_len)

        # Label tensor
        y = torch.LongTensor(self.bsz)
        y1s = torch.LongTensor(self.bsz)
        y1e = torch.LongTensor(self.bsz)
        y2s = torch.LongTensor(self.bsz)
        y2e = torch.LongTensor(self.bsz)

        while True:
            if self.example_ptr >= len(self.examples):
                break
            start_id = self.example_ptr
            cur_bsz = min(self.bsz, len(self.examples) - start_id)
            cur_batch = self.examples[start_id: start_id + cur_bsz]
            cur_batch.sort(key=lambda x: len(x["question"]), reverse=True)

            ids = []
            for mapping in [context_mask]:
                mapping.zero_()

            max_c_len = 0
            ori_text = []
            for k in range(len(cur_batch)):
                question = cur_batch[k]["question"]
                ids.append(cur_batch[k]["_id"])

                query_tokens = ["[CLS]"] + tokenizer.tokenize(question)
                if len(query_tokens) > self.max_len - 1:
                    query_tokens = query_tokens[:self.max_len - 1]
                query_tokens.append("[SEP]")
                ori_text.append(query_tokens)

                q_len = len(query_tokens)
                max_c_len = max([max_c_len, q_len])
                input_ids = tokenizer.convert_tokens_to_ids(query_tokens)+ [0]*(self.max_len - q_len)

                context_idxs[k].copy_(torch.Tensor(input_ids))
                context_mask[k][:q_len] = 1
                y1s[k],y1e[k],y2s[k],y2e[k],y[k] = 0,0,0,0,0
                if cur_batch[k]['evidences'] != []:
                    relations = []
                    for e in cur_batch[k]['evidences']:
                        if e[1] not in relations:
                            relations.append(e[1])
                    y[k] = Q_type.get(cur_batch[k]["type"],0)
                    input_str = " ".join(query_tokens)
                    if len(relations)>0:
                        ans = " ".join(tokenizer.tokenize(relations[0]))
                        y1s[k],y1e[k] = find_pos(input_str,ans)
                    if len(relations)>1:
                        ans = " ".join(tokenizer.tokenize(relations[1]))
                        y2s[k],y2e[k] = find_pos(input_str,ans)

            self.example_ptr += cur_bsz

            yield {
                'context_idxs': context_idxs[:cur_bsz, :max_c_len].contiguous(),
                'context_mask': context_mask[:cur_bsz, :max_c_len].contiguous(),
                'y': y[:cur_bsz],
                'y1s': y1s[:cur_bsz],
                'y2s': y2s[:cur_bsz],
                'y1e': y1e[:cur_bsz],
                'y2e': y2e[:cur_bsz],
                'ids': ids,
                'ori_text': ori_text}



def predict(model, dataloader, prediction_file, is_test = True):
    model.eval()
    answer_dict = {}
    if not is_test:
      t_acc, r_acc = [],[]
    dataloader.refresh()
    for batch in tqdm(dataloader, desc="Iteration"):
        batch = {k:v.cuda() if k not in ['ids','ori_text'] else v for k,v in batch.items()}
        t_logits, r1s,r1e,r2s,r2e = model.forward(batch)

        t, r1s,r1e,r2s,r2e = np.argmax(t_logits.data.cpu().numpy(), 1), r1s.data.cpu().numpy(), r1e.data.cpu().numpy(), r2s.data.cpu().numpy(), r2e.data.cpu().numpy()

        for i in range(t.shape[0]):

            cur_id = batch['ids'][i]
            if not is_test:
              t_acc.append(1 if t[i]==batch['y'][i] else 0)
            ori_text = batch['ori_text'][i]
            ori_ans1 = " ".join(ori_text[batch['y1s'][i]:batch['y1e'][i]+1]).replace(" ##","").replace("##","")
            ori_ans2 = " ".join(ori_text[batch['y2s'][i]:batch['y2e'][i]+1]).replace(" ##","").replace("##","")

            y1s,y1e,y2s,y2e = int(batch['y1s'][i]),int(batch['y1e'][i]),int(batch['y2s'][i]),int(batch['y2e'][i])
            if int(t[i]) == 2:
                start,end = int(np.argmax(r1s[i]+r2s[i])),int(np.argmax(r1e[i]+r2e[i]))
                if not is_test:
                  f1,_,_ = F1(start,end,batch['y1s'][i],batch['y1e'][i])
                  r_acc += [f1,f1]
                ans = " ".join(ori_text[start:end+1]).replace(" ##","").replace("##","") if start<=end else ""
                answer_dict[cur_id] = {'Type':int(t[i]),'R':[ans],'answers':[ans,ori_ans1],'spans':[start,end,y1s,y1e]}
            else:
                l1, r1 = int(np.argmax(r1s[i])),int(np.argmax(r1e[i]))
                if not is_test:
                  f1,_,_ = F1(l1,r1,batch['y1s'][i],batch['y1e'][i])
                  r_acc.append(f1)
                ans1 = " ".join(ori_text[l1:r1+1]).replace(" ##","").replace("##","") if l1<=r1 else ""

                l2, r2 = int(np.argmax(r2s[i])),int(np.argmax(r2e[i]))
                if not is_test:
                  f1,_,_ = F1(l2,r2,batch['y2s'][i],batch['y2e'][i])
                  r_acc.append(f1)
                ans2 = " ".join(ori_text[l2:r2+1]).replace(" ##","").replace("##","") if l2<=r2 else ""
                answer_dict[cur_id] = {'Type':int(t[i]),'R':[ans1,ans2],'answers':[ans1,ori_ans1,ans2,ori_ans2],'spans':[[l1,r1,y1s,y1e],[l2,r2,y2s,y2e]]}
    if not is_test:
      scores = []
      scores.append('T-acc: {:.3f}\t R_f1:{:.3f}\n'.format(sum(t_acc)/len(t_acc),sum(r_acc)/len(r_acc)))
      print(scores)
    with open(prediction_file, 'w') as f:
        json.dump(answer_dict, f, indent=2)

    model.train()


if __name__ == "__main__":
    gradient_accumulation_steps = 1
    batch_size = 32
    max_len = 128
    name = 'SRE'
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    os.environ["CUDA_VISIBLE_DEVICES"] = '0'

    # Set GPU Issue
    n_gpu = torch.cuda.device_count()
    batch_size = int(batch_size / gradient_accumulation_steps)

    logger.info("n_gpu: {} Grad_Accum_steps: {} Batch_size: {}".format(n_gpu,gradient_accumulation_steps, batch_size))

    # Set Seeds
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(42)

    # Set datasets
    # Full_Loader = DataIterator(args.train_file, args)
    # Subset_Loader = DataIterator(args.train_file, args, subset=True)
    Full_Loader = DataIterator('dev.json', batch_size, max_len, sequential=True)
    test_dataset = DataIterator('test.json', batch_size, max_len, sequential=True)

    # Prepare Model

    model = BertForTypeSpanComp.from_pretrained('bert-base-uncased')
    if n_gpu > 1:
        model = torch.nn.DataParallel(model)
        model = model.module
    model.cuda()

    # Prepare Optimizer
    lr = 2e-5
    epochs = 15
    warmup_proportion =  0.1
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    t_total = int(len(Full_Loader)*epochs/batch_size/gradient_accumulation_steps)

    global_step = 1
    if 1:
        model.train()
        for epc in trange(int(epochs), desc="Epoch"):
            tr_loss, r1_loss, r2_loss = 0, 0, 0
            Loader = Full_Loader
            Loader.refresh()

            for step, batch in enumerate(tqdm(Loader, desc="Iteration")):
                batch = {k:v.cuda() if k not in ['ids','ori_text'] else v for k,v in batch.items()}
                loss, loss1, loss2 = model.forward(batch, is_train = True)
                if n_gpu > 1:
                    loss = loss.mean()  # mean() to average on multi-gpu.
                    loss1 = loss1.mean()
                    loss2 = loss2.mean()
                if gradient_accumulation_steps > 1:
                    loss = loss /gradient_accumulation_steps
                    loss1 = loss1 /gradient_accumulation_steps
                    loss2 = loss2 /gradient_accumulation_steps
                loss.backward()

                tr_loss += loss.item()
                r1_loss += loss1.item()
                r2_loss += loss2.item()
                if (step + 1) % gradient_accumulation_steps == 0:
                    # modify learning rate with special warm up BERT uses
                    lr_this_step = lr * warmup_linear(global_step / t_total, warmup_proportion)
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_this_step
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1


                if global_step % 1000 == 0 and step>0:
                    loss_log = 'total:{}\t R1:{}\t R2:{}'.format(tr_loss/step,r1_loss/step,r2_loss/step)

            # Save a trained model
            checkpoint_path = os.path.join(name, 'model')
            prediction_path = os.path.join(name, 'predict')
            best_model = os.path.join(checkpoint_path, "best_model.pth")

            os.makedirs(checkpoint_path, exist_ok=True)
            os.makedirs(prediction_path, exist_ok=True)

            torch.save(model.state_dict(), os.path.join(checkpoint_path, "ckpt_epoch_{}.pth".format(epc)))
            predict(model, test_dataset, os.path.join(prediction_path, 'pred_epoch_{}.json'.format(epc)))


Streaming output truncated to the last 5000 lines.
Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.10it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.11it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.12it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.12it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.12it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.13it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.13it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.14it/s]

Iteration: 100%|██████████| 393/393 [01:35<00:00,  4.14it/s]

Epoch: 100%|██████████| 15/15 [33:42<00:00, 134.80s/it]
